In [9]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
# import seaborn as sns

from t import *


%matplotlib inline
sns.set_style("whitegrid")

# df= pd.read_csv('/data/DS_INTERN/data/RAW_DATA/train_test_data.csv', low_memory=False)
df = pd.read_csv('/data/DS_INTERN/data/RAW_DATA/train_test_data_new.csv', low_memory=False)


df1 = add_target_column(df)

# train_df , test_df = train_test_split(df1)
train_df = pd.read_csv('train_df_augmented.csv')
test_df = get_test_data(df1)

train_df = perform_preprocessing(train_df)
test_df = perform_preprocessing(test_df)

/home/jupyter/t.py:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['FEAT1'] = pd.Series(remove_special_characters(df['FEAT1']), index=df.index)
/home/jupyter/t.py:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['FEAT1'] = pd.Series(perform_spell_correction_manual(df['FEAT1']), index=df.index)
/home/jupyter/t.py:110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pand

In [10]:
len(train_df)

69937

In [11]:
X_train = train_df['FEAT1']
Y_train = train_df['TARGET']
X_test = test_df['FEAT1']

In [12]:
def pipeline_LR_HP(max_iter):
    
    from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
    
    from sklearn.linear_model import LogisticRegression
    from sklearn.pipeline import Pipeline


    pipe = Pipeline([('vect',TfidfVectorizer(min_df=10)),
                    ('clf', LogisticRegression(C=250, penalty='l2',solver='lbfgs',class_weight='balanced',max_iter= max_iter,
                                              random_state=42))])
    
    return pipe

In [13]:
pipe = pipeline_voting_hard(solver='lbfgs',penalty='l2',random_state=42,max_iter=100)

In [14]:
%%time
pipe.fit(X_train, Y_train)

CPU times: user 7min 43s, sys: 6min 33s, total: 14min 16s
Wall time: 7min


/home/jupyter/libraries/nb_env/lib64/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


VotingClassifier(estimators=[('RF',
                              Pipeline(steps=[('vect',
                                               TfidfVectorizer(min_df=10)),
                                              ('clf',
                                               RandomForestClassifier(class_weight='balanced',
                                                                      n_estimators=220,
                                                                      random_state=42))])),
                             ('LR',
                              Pipeline(steps=[('vect',
                                               TfidfVectorizer(min_df=10)),
                                              ('clf',
                                               LogisticRegression(C=250,
                                                                  class_weight='balanced',
                                                                  random_state=42))]))])

In [15]:
Y_pred = pipe.predict(X_test)

In [16]:
test_df['Predicted_target'] = pd.Series(Y_pred, index= test_df.index)

/home/jupyter/libraries/nb_env/lib64/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [17]:
test_df = trust_factor(test_df)
test_df.describe()

/libs/base/pandas/core/frame.py:2540: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]
/home/jupyter/t.py:681: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  test_df['category'] = pd.Series(perform_spell_correction_manual(test_df['category']), index=test_df.index)
/home/jupyter/t.py:682: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.

,MDS_FAM_ID,UPC_NBR,DEPT_NBR,SALES_AMT_DOLLAR,NET_SHIP_COST_DOLLAR,SALES_QTY,CONSUMER_ITEM_NBR,REPL_GROUP_NBR,PRODUCT_NBR,fuzzy_category,fuzzy_variety,TF_cat,TF_var,TF
count,2.441870e+05,2.441870e+05,244187.000000,8.654800e+04,5.211600e+04,8.654800e+04,2.429830e+05,2.330900e+05,2.441340e+05,244187.000000,244187.000000,244187.000000,244187.000000,244187.000000
mean,5.422204e+07,1.193594e+11,92.178691,1.467326e+05,2.199840e+05,6.414759e+04,9.569301e+06,1.031263e+08,9.562269e+06,82.234394,89.052067,0.692993,0.798863,0.860889
std,1.766833e+07,1.866042e+11,9.161114,1.776146e+06,2.169458e+06,6.511948e+05,6.049217e+06,7.641933e+07,6.071109e+06,27.603842,22.516637,0.461253,0.400851,0.346063
min,3.871000e+03,0.000000e+00,1.000000,-2.769620e+03,-2.889000e+04,-3.154176e+04,2.394000e+03,1.500000e+01,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5.310606e+07,4.002031e+10,94.000000,6.867275e+00,1.029158e+03,3.000000e+00,8.197535e+06,1.547112e+07,8.080944e+06,60.000000,100.000000,0.000000,1.000000,1.000000
50%,5.467994e+07,4.002102e+10,94.000000,9.378376e+02,6.548292e+03,3.523700e+02,9.505364e+06,1.336468e+08,9.548793e+06,100.000000,100.000000,1.000000,1.000000,1.000000
75%,5.505387e+07,8.548690e+10,94.000000,1.285656e+04,3.781332e+04,6.486000e+03,1.065362e+07,1.835725e+08,1.065386e+07,100.000000,100.000000,1.000000,1.000000,1.000000
max,2.558749e+08,8.002970e+12,95.000000,1.884090e+08,1.914145e+08,6.278640e+07,1.164568e+08,2.067939e+08,1.164568e+08,100.000000,100.000000,1.000000,1.000000,1.000000


In [18]:
test_df1 = pd.read_csv('test_df_with_model_predictions_with_fuzzy.csv', low_memory=False)

In [19]:
test_df1['Predicted_target'] = pd.Series(Y_pred, index= test_df1.index)

In [20]:
test_df1['Final_Predicted_target'] = np.where((test_df1['Ratio'] >= 70), test_df1['fuzzy_target'], test_df1['Predicted_target'])

In [21]:
test_df1

,Unnamed: 0,COUNTRY_CODE,SBU,SP_CLASS,MDS_FAM_ID,UPC_NBR,DEPT_NBR,DEPT_DESC,CATG_DESC,SUBCATG_DESC,...,Ratio,fuzzy_target,Final_Predicted_target,category,variety,fuzzy_category,fuzzy_variety,TF_cat,TF_var,TF
0,0,US,FOOD,PRODUCE & FLOWERS,254412860,76101011282,94,PRODUCE,NUT MEAT,NUT WALNUT,...,63,NUT MEATS_SHELLED WALNUTS,NUT MEATS_SHELLED WALNUTS,NUT MEAT,SHELLED WALNUT,100,100,1,1,1
1,1,US,FOOD,PRODUCE & FLOWERS,162040632,80394430117,94,PRODUCE,MISC FLORAL,PR FLORAL,...,59,DRIED FRUIT_DRIED MANGOES,DRIED VEGGIES_DRIED SNAP PEAS,DRIED VEGGIE,DRIED SNAP PEA,100,100,1,1,1
2,2,US,FOOD,PRODUCE & FLOWERS,93643850,71631063457,94,PRODUCE,MISC FLORAL,HI AND MORE,...,58,CUT FRUIT PROCESSED_SEASONAL BLEND,VARIETY TROPICALS_TROPICAL FRUIT MISC,APPLE,MISC,40,100,0,1,1
3,3,US,FOOD,PRODUCE & FLOWERS,93241707,71631063457,94,PRODUCE,MISC FLORAL,HI AND MORE,...,58,CUT FRUIT PROCESSED_SEASONAL BLEND,VARIETY TROPICALS_TROPICAL FRUIT MISC,APPLE,MISC,40,100,0,1,1
4,4,US,FOOD,PRODUCE & FLOWERS,75494753,8185195601,94,PRODUCE,MISC FLORAL,HI AND MORE,...,62,MIXED BOUQUETS_MIXED BOUQUETS,VARIETY TROPICALS_TROPICAL FRUIT MISC,APPLE,MISC,40,100,0,1,1
5,5,US,FOOD,PRODUCE & FLOWERS,90953283,81305501187,94,PRODUCE,MISC FLORAL,HI AND MORE,...,66,CUT FRUIT PROCESSED_SEASONAL BLEND,VARIETY TROPICALS_TROPICAL FRUIT MISC,APPLE,MISC,40,100,0,1,1
6,6,US,FOOD,PRODUCE & FLOWERS,80575903,7102231127,94,PRODUCE,MISC FLORAL,HI AND MORE,...,56,MIXED BOUQUETS_MIXED BOUQUETS,VARIETY TROPICALS_TROPICAL FRUIT MISC,APPLE,MISC,40,100,0,1,1
7,7,US,FOOD,PRODUCE & FLOWERS,12280720,2491912010,94,PRODUCE,MISC FLORAL,HI AND MORE,...,63,CUT FRUIT PROCESSED_FRUIT BOWL,CUT FRUIT PROCESSED_FRUIT TRAYS,APPLE,MISC,100,100,1,1,1
8,8,US,FOOD,PRODUCE & FLOWERS,90953278,81305501186,94,PRODUCE,MISC FLORAL,HI AND MORE,...,66,CUT FRUIT PROCESSED_SEASONAL BLEND,VARIETY TROPICALS_TROPICAL FRUIT MISC,APPLE,MISC,40,100,0,1,1
9,9,US,FOOD,PRODUCE & FLOWERS,109240313,81824002024,94,PRODUCE,MISC FLORAL,HI AND MORE,...,68,MIXED BOUQUETS_MIXED BOUQUETS,CONSUMER BUNCHES_CONSUMER BUNCHES,CONSUMER BUNCH,CONSUMER BUNCH,50,50,0,0,0


In [22]:
def trust_factor(test_df):
    
    import Levenshtein as lev
    from fuzzywuzzy import fuzz
    
#    test_df['Predicted_target'] = pd.Series(Y_pred, index= test_df.index)
    test_df[['category','variety']] = test_df['Final_Predicted_target'].str.split('_', expand=True)
#    test_df.drop(['Predicted_target'], axis=  1, inplace = True)
    
    test_df['category'] = pd.Series(perform_spell_correction_manual(test_df['category']), index=test_df.index)
    test_df['variety'] = pd.Series(perform_spell_correction_manual(test_df['variety']), index=test_df.index)
    
    test_df['category'] = pd.Series(perform_spell_correction_walmart(test_df['category']), index=test_df.index)
    test_df['variety'] = pd.Series(perform_spell_correction_walmart(test_df['variety']), index=test_df.index)
    
    
    test_df['category'] = pd.Series(perform_lemmatization(test_df['category']), index=test_df.index)
    test_df['variety'] = pd.Series(perform_lemmatization(test_df['variety']), index=test_df.index)
    
    
    test_df['fuzzy_category'] = test_df.apply(lambda x: fuzz.partial_token_set_ratio(x['FEAT1'],x['category']), axis=1)
    test_df['fuzzy_variety'] = test_df.apply(lambda x: fuzz.partial_token_set_ratio(x['FEAT1'],x['variety']), axis=1)
    
    test_df['TF_cat'] = test_df.apply(lambda x: 1 if x['fuzzy_category']>=80 else 0, axis=1)
    test_df['TF_var'] = test_df.apply(lambda x: 1 if x['fuzzy_variety']>=80 else 0, axis=1)
    
    test_df['TF'] = np.where((test_df['TF_cat'] == 1) | (test_df['TF_var'] ==1),1, 0)
    
    return test_df

In [23]:
test_df_final = trust_factor(test_df1)
test_df_final.describe()

,Unnamed: 0,MDS_FAM_ID,UPC_NBR,DEPT_NBR,REVISED_CATEGORY,REVISED_VARIETY,SALES_AMT_DOLLAR,NET_SHIP_COST_DOLLAR,SALES_QTY,CONSUMER_ITEM_NBR,REPL_GROUP_NBR,PRODUCT_NBR,TARGET,Ratio,fuzzy_category,fuzzy_variety,TF_cat,TF_var,TF
count,244187.000000,2.441870e+05,2.441870e+05,244187.000000,0.0,0.0,8.654800e+04,5.211600e+04,8.654800e+04,2.429830e+05,2.330900e+05,2.441340e+05,0.0,244187.000000,244187.000000,244187.000000,244187.000000,244187.000000,244187.000000
mean,122093.000000,5.422204e+07,1.193594e+11,92.178691,NaN,NaN,1.467326e+05,2.199840e+05,6.414759e+04,9.569301e+06,1.031263e+08,9.562269e+06,NaN,59.887144,82.548334,89.169804,0.698252,0.800849,0.862577
std,70490.859429,1.766833e+07,1.866042e+11,9.161114,NaN,NaN,1.776146e+06,2.169458e+06,6.511948e+05,6.049217e+06,7.641933e+07,6.071109e+06,NaN,8.797121,27.453122,22.410478,0.459018,0.399362,0.344294
min,0.000000,3.871000e+03,0.000000e+00,1.000000,NaN,NaN,-2.769620e+03,-2.889000e+04,-3.154176e+04,2.394000e+03,1.500000e+01,0.000000e+00,NaN,16.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,61046.500000,5.310606e+07,4.002031e+10,94.000000,NaN,NaN,6.867275e+00,1.029158e+03,3.000000e+00,8.197535e+06,1.547112e+07,8.080944e+06,NaN,54.000000,60.000000,100.000000,0.000000,1.000000,1.000000
50%,122093.000000,5.467994e+07,4.002102e+10,94.000000,NaN,NaN,9.378376e+02,6.548292e+03,3.523700e+02,9.505364e+06,1.336468e+08,9.548793e+06,NaN,59.000000,100.000000,100.000000,1.000000,1.000000,1.000000
75%,183139.500000,5.505387e+07,8.548690e+10,94.000000,NaN,NaN,1.285656e+04,3.781332e+04,6.486000e+03,1.065362e+07,1.835725e+08,1.065386e+07,NaN,64.000000,100.000000,100.000000,1.000000,1.000000,1.000000
max,244186.000000,2.558749e+08,8.002970e+12,95.000000,NaN,NaN,1.884090e+08,1.914145e+08,6.278640e+07,1.164568e+08,2.067939e+08,1.164568e+08,NaN,100.000000,100.000000,100.000000,1.000000,1.000000,1.000000


In [26]:
test_df_final.to_csv('test_df_LR_with_fuzzy')

In [27]:
test_df_final.to_csv('/data/DS_INTERN/data/RAW_DATA/test_df_LR_with_fuzzy.csv')

In [32]:
tiku = pd.read_csv('test_df_with_model_predictions_with_fuzzy.csv', low_memory=False)

In [34]:
tiku.describe()

,Unnamed: 0,MDS_FAM_ID,UPC_NBR,DEPT_NBR,REVISED_CATEGORY,REVISED_VARIETY,SALES_AMT_DOLLAR,NET_SHIP_COST_DOLLAR,SALES_QTY,CONSUMER_ITEM_NBR,REPL_GROUP_NBR,PRODUCT_NBR,TARGET,Ratio,fuzzy_category,fuzzy_variety,TF_cat,TF_var,TF
count,244187.000000,2.441870e+05,2.441870e+05,244187.000000,0.0,0.0,8.654800e+04,5.211600e+04,8.654800e+04,2.429830e+05,2.330900e+05,2.441340e+05,0.0,244187.000000,244187.000000,244187.000000,244187.000000,244187.000000,244187.000000
mean,122093.000000,5.422204e+07,1.193594e+11,92.178691,NaN,NaN,1.467326e+05,2.199840e+05,6.414759e+04,9.569301e+06,1.031263e+08,9.562269e+06,NaN,59.887144,81.655264,89.202148,0.683644,0.803212,0.850578
std,70490.859429,1.766833e+07,1.866042e+11,9.161114,NaN,NaN,1.776146e+06,2.169458e+06,6.511948e+05,6.049217e+06,7.641933e+07,6.071109e+06,NaN,8.797121,27.850925,22.532911,0.465055,0.397571,0.356505
min,0.000000,3.871000e+03,0.000000e+00,1.000000,NaN,NaN,-2.769620e+03,-2.889000e+04,-3.154176e+04,2.394000e+03,1.500000e+01,0.000000e+00,NaN,16.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,61046.500000,5.310606e+07,4.002031e+10,94.000000,NaN,NaN,6.867275e+00,1.029158e+03,3.000000e+00,8.197535e+06,1.547112e+07,8.080944e+06,NaN,54.000000,60.000000,100.000000,0.000000,1.000000,1.000000
50%,122093.000000,5.467994e+07,4.002102e+10,94.000000,NaN,NaN,9.378376e+02,6.548292e+03,3.523700e+02,9.505364e+06,1.336468e+08,9.548793e+06,NaN,59.000000,100.000000,100.000000,1.000000,1.000000,1.000000
75%,183139.500000,5.505387e+07,8.548690e+10,94.000000,NaN,NaN,1.285656e+04,3.781332e+04,6.486000e+03,1.065362e+07,1.835725e+08,1.065386e+07,NaN,64.000000,100.000000,100.000000,1.000000,1.000000,1.000000
max,244186.000000,2.558749e+08,8.002970e+12,95.000000,NaN,NaN,1.884090e+08,1.914145e+08,6.278640e+07,1.164568e+08,2.067939e+08,1.164568e+08,NaN,100.000000,100.000000,100.000000,1.000000,1.000000,1.000000


In [30]:
from t import *

In [31]:
tiku_t = trust_factor(tiku)
tiku_t.describe()

,Unnamed: 0,MDS_FAM_ID,UPC_NBR,DEPT_NBR,REVISED_CATEGORY,REVISED_VARIETY,SALES_AMT_DOLLAR,NET_SHIP_COST_DOLLAR,SALES_QTY,CONSUMER_ITEM_NBR,REPL_GROUP_NBR,PRODUCT_NBR,TARGET,fuzzy_category,fuzzy_variety,TF_cat,TF_var,TF
count,244187.000000,2.441870e+05,2.441870e+05,244187.000000,0.0,0.0,8.654800e+04,5.211600e+04,8.654800e+04,2.429830e+05,2.330900e+05,2.441340e+05,0.0,244187.000000,244187.000000,244187.000000,244187.00000,244187.000000
mean,171868.691196,5.422204e+07,1.193594e+11,92.178691,NaN,NaN,1.467326e+05,2.199840e+05,6.414759e+04,9.569301e+06,1.031263e+08,9.562269e+06,NaN,81.296232,89.046722,0.677530,0.80060,0.848051
std,71410.376483,1.766833e+07,1.866042e+11,9.161114,NaN,NaN,1.776146e+06,2.169458e+06,6.511948e+05,6.049217e+06,7.641933e+07,6.071109e+06,NaN,28.002008,22.661125,0.467423,0.39955,0.358972
min,6537.000000,3.871000e+03,0.000000e+00,1.000000,NaN,NaN,-2.769620e+03,-2.889000e+04,-3.154176e+04,2.394000e+03,1.500000e+01,0.000000e+00,NaN,0.000000,0.000000,0.000000,0.00000,0.000000
25%,111294.500000,5.310606e+07,4.002031e+10,94.000000,NaN,NaN,6.867275e+00,1.029158e+03,3.000000e+00,8.197535e+06,1.547112e+07,8.080944e+06,NaN,50.000000,100.000000,0.000000,1.00000,1.000000
50%,172341.000000,5.467994e+07,4.002102e+10,94.000000,NaN,NaN,9.378376e+02,6.548292e+03,3.523700e+02,9.505364e+06,1.336468e+08,9.548793e+06,NaN,100.000000,100.000000,1.000000,1.00000,1.000000
75%,233387.500000,5.505387e+07,8.548690e+10,94.000000,NaN,NaN,1.285656e+04,3.781332e+04,6.486000e+03,1.065362e+07,1.835725e+08,1.065386e+07,NaN,100.000000,100.000000,1.000000,1.00000,1.000000
max,294434.000000,2.558749e+08,8.002970e+12,95.000000,NaN,NaN,1.884090e+08,1.914145e+08,6.278640e+07,1.164568e+08,2.067939e+08,1.164568e+08,NaN,100.000000,100.000000,1.000000,1.00000,1.000000


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [8]:
def pipeline_LR_HP(max_iter):
    
    from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
    
    from sklearn.linear_model import LogisticRegression
    from sklearn.pipeline import Pipeline

    pipe = Pipeline([('vect',TfidfVectorizer(min_df=10)),
#                     ('scaler',StandardScaler(with_mean=False)),
                    ('clf', LogisticRegression(C=250,solver='saga',class_weight='balanced',max_iter= max_iter))])
    
    return pipe

In [9]:
pipe = pipeline_LR_HP(100)
pipe1 = pipeline_LR_HP(100)

In [14]:
%%time

pipe.fit(X_train, Y_train)
pipe1.fit(X_train, Y_train)

/home/jupyter/libraries/nb_env/lib64/python3.6/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


CPU times: user 5min 28s, sys: 416 ms, total: 5min 28s
Wall time: 5min 28s


/home/jupyter/libraries/nb_env/lib64/python3.6/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [15]:
Y_pred = pipe.predict(X_test)
Y_pred1 = pipe1.predict(X_test)

In [16]:
pd.Series(Y_pred).value_counts().head(5)

VARIETY TROPICALS_BREADFRUIT            51922
MIXED VEGETABLES_RHUBARB                 9648
MELONS_CASABA                            9018
CITRUS_ORANGES - OTHER                   7430
CUT FRUIT PROCESSED_CUT FRUIT CITRUS     5908
dtype: int64

In [18]:
pd.Series(Y_pred1).value_counts().head(10)

VARIETY TROPICALS_BREADFRUIT             42833
MELONS_CASABA                             9409
MIXED VEGETABLES_RHUBARB                  8302
CITRUS_ORANGES - OTHER                    7087
VARIETY TROPICALS_TROPICAL FRUIT MISC     6087
CUT FRUIT PROCESSED_CUT FRUIT CITRUS      6016
APPLES_MISC                               4898
MUSHROOMS_MUSHROOMS VARIETY               4551
CITRUS_GRAPEFRUIT                         4094
PEARS_PEAR MISC                           4033
dtype: int64

In [1]:
import fuzzywuzzy

In [2]:
import numpy as np
import pandas as pd

In [3]:
test = pd.read_csv('test_df_LR_with_fuzzy', low_memory=False)

In [6]:
test[test.Ratio <70].iloc[0]

Unnamed: 0                                                                0
Unnamed: 0.1                                                              0
COUNTRY_CODE                                                             US
SBU                                                                    FOOD
SP_CLASS                                                  PRODUCE & FLOWERS
MDS_FAM_ID                                                        254412860
UPC_NBR                                                         76101011282
DEPT_NBR                                                                 94
DEPT_DESC                                                           PRODUCE
CATG_DESC                                                          NUT MEAT
SUBCATG_DESC                                                     NUT WALNUT
FINELINE_DESC                                                   WALNUT TRAY
SIGNING_DESC                                                    MERCHANDISE
ITEM1_DESC  